# Install Spark Google Colab


---

In [ ]:
# Connecting Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# Setting up PySpark in Colab
# 1. First task is to download Java.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# 2. Install Apache Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

!tar -xzvf /content/spark-3.4.0-bin-hadoop3.tgz

# 3. Install and that is the findspark library. It will locate Spark on the system and import it as a regular library.
!pip install -q findspark

# 4. set the environment path. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

# 5. We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.
import findspark
findspark.init('spark-3.4.0-bin-hadoop3')

# 6. Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

!rm /content/spark-3.4.0-bin-hadoop3.tgz

# ETL - Api Football

In [ ]:
# Install packages
!pip install pandas
!pip install numpy
!pip install pprintpp
!pip install requests

In [ ]:
# Import packages
import requests
import json
import pandas as pd

In [ ]:
# docs
# https://rapidapi.com/api-sports/api/api-football/
# https://www.api-football.com/documentation-v3

# Save leagues into a json file
url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

headers = {
	"X-RapidAPI-Key": "81016ac729mshd42ae2fc25f8821p107a85jsn3fb584730546",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers).json()

with open("leagues.json", "w") as write_file:
    json.dump(response, write_file)

In [ ]:
# Load json file into leagues variable
with open('leagues.json','r') as leagues_file:
  leagues = json.load(leagues_file)

league_list = []
for league in leagues['response']:
  league_id = league['league']['id']
  league_name = league['league']['name']
  league_element = {
      'league_id' : league_id,
      'league_name' :league_name
  }

  league_list.append(league_element)

league_df = pd.DataFrame.from_dict(league_list)

league_df

In [ ]:
url = "https://api-football-v1.p.rapidapi.com/v3/teams/countries"

headers = {
	"X-RapidAPI-Key": "81016ac729mshd42ae2fc25f8821p107a85jsn3fb584730546",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

pprint.pprint(response.json())